In [129]:
# Importamos bibliotecas
import requests
from lxml import html
import time
import random
import pandas as pd
import json

In [130]:
#macros, se podren pasar como argumentos en futuras versiones
#hasta = '2021-12-31'
#desde = '2021-01-01'
# area = 'advantage actor critic'
# fuente = 'arxiv'
# numero de paginas a revisar (paginas de 10 elementos)
num_de_paginas = 2


La clase primaria de springer no es correcto, tiene un html generico para que se ejecute sin dar fallo
tengo que hacer que colocar el html correcto
Cuando google te banea el bucle for que itera las pagina especificas falla --arreglar
El .json solo guarda los datos de la ultima iteracion --arreglas


In [131]:
info_fuentes = []

info_fuente_arvix = {
    'area':'advantage actor critic',
    'fuente':'arxiv',
    'titulo':'//h1 [@class ="title mathjax"]/text()',
    'autores':'//div [@class ="authors"]/a/text()',
    'abstract':'//blockquote [@class ="abstract mathjax"]/text()',
    'clase_primaria':'//span [@class ="primary-subject"]/text()',
    # 'otras_clases':'//td [@class ="tablecell subjects"]/text()',
}
info_fuente_arvix['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_arvix.get("area").replace(" ", "+")+'+source%3A'+info_fuente_arvix.get("fuente")+'&oq='
info_fuentes.append(info_fuente_arvix)

info_fuente_springer = {
    'area':'advantage actor critic',
    'fuente':'springer',
    'titulo':'//h1 [@class ="c-article-title"]/text()',
    'autores':'//li [@class ="c-article-author-list__item"]/a/text()',
    'abstract':'//div [@class ="c-article-section__content"][@id="Abs1-content"]/p/text()',
    'clase_primaria':'//li [@class ="c-article-subject-list__subject"]/span/text()',
    # 'otras_clases':'',
}
info_fuente_springer['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_springer.get("area").replace(" ", "+")+'+source%3A'+info_fuente_springer.get("fuente")+'&oq='
info_fuentes.append(info_fuente_springer)

info_fuente_SSRN = {
    'area':'advantage actor critic',
    'fuente':'SSRN',
    'titulo':'//h1/text()',
    'autores':'//div [@class ="authors authors-full-width"]/h2/a/text()',
    'abstract':'//div [@class ="abstract-text"]/p/text()',
    'clase_primaria':'//vacio',
    # 'otras_clases':'',
}
info_fuente_SSRN['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_SSRN.get("area").replace(" ", "+")+'+source%3A'+info_fuente_SSRN.get("fuente")+'&oq='
info_fuentes.append(info_fuente_SSRN)

info_fuente_cambridge = {
    'area':'machine learning',
    'fuente':'cambridge',
    'titulo':'//h1 [@data-v-0950e612=""]/text()',
    'autores':'//div [@class="row contributor-type"]/div/a [@class="app-link app-link__text app-link--accent"]/span/text()',
    'abstract':'//div [@class="abstract-content"]/div/div/p/text()',
    'clase_primaria':'//div [@class="row keywords__pills"]/a/span/text()',
    # 'otras_clases':'',
}
info_fuente_cambridge['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_cambridge.get("area").replace(" ", "+")+'+source%3A'+info_fuente_cambridge.get("fuente")+'&oq='
info_fuentes.append(info_fuente_cambridge)

In [132]:
for k in range(len(info_fuentes)):
    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'Chrome/96.0.4664.93',
        }
    )

    req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
    if req.status_code == 429:
        print('wait' +req.headers.get('Retry-After'))
        time.sleep(int(req.headers.get('Retry-After'))+1)
        req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
    print(req)

    #Convertimos a tree 
    arbol_html = html.fromstring(req.content)

    # start=10 

    papers = []


    for i in range (num_de_paginas):
        print('Empiezo vuelta '+str(i)+' :'+time.strftime("%H:%M:%S", time.gmtime()))
        req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
        print('scrapig google', info_fuentes[k].get('pagina'))

        # repetir despues del tiempo de espera
        if req.status_code == 429:
            print('wait' +req.headers.get('Retry-After'))
            time.sleep(int(req.headers.get('Retry-After'))+1)
            req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)

        # Convertimos a tree 
        arbol_html = html.fromstring(req.content)
        temp_res = arbol_html.xpath('//h3 [@class ="gs_rt"]/a/@href')

        # for j in range(len(temp_res)):
        for j in range(1):
            paper_dic = {}
            print('scraping a', temp_res[j])
            pag = temp_res[j]
            time.sleep(random.randint(1,2)*0.5)
            paper_res =  requests.get(pag, headers=headers)
            
            if paper_res.status_code == 429:
                print('wait' +paper_res.headers.get('Retry-After'))
                time.sleep(int(paper_res.headers.get('Retry-After'))+1)
                paper_res =  requests.get(pag, headers=headers)
            paper_arbol_html = html.fromstring(paper_res.content)

            # especifico de la pagina
            paper_dic['fuente'] = info_fuentes[k]['fuente']
            paper_dic['titulo'] = paper_arbol_html.xpath(info_fuentes[k]['titulo'])
            paper_dic['autores'] = paper_arbol_html.xpath(info_fuentes[k]['autores'])
            paper_dic['abstract'] = paper_arbol_html.xpath(info_fuentes[k]['abstract'])
            paper_dic['clase_primaria'] = paper_arbol_html.xpath(info_fuentes[k]['clase_primaria'])
            # paper_dic['otras_clases'] = paper_arbol_html.xpath(info_fuentes[k]['otras_clases'])
            #metadata
            paper_dic['pag_espec'] = pag
            paper_dic['pag_espec'] = info_fuentes[k].get('pagina')
            #generico
            paper_dic["n_citaciones"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/text()')
            paper_dic["citado"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/@href')
            papers.append(paper_dic)
        info_fuentes[k]['pagina'] = 'https://scholar.google.com/scholar?start='+str(10*(i+1))+'&q='+info_fuentes[k].get("area").replace(" ", "+")+'+source%3A'+info_fuentes[k].get("fuente").split('.')[0]+'&oq='
        
#guarda la info es .json
with open('papers.json', 'a') as file:
    json.dump(papers, indent=7, fp=file)
    
    
    


<Response [200]>
Empiezo vuelta 0 :10:57:38
scrapig google https://scholar.google.com/scholar?q=advantage+actor+critic+source%3Aarxiv&oq=
scraping a https://arxiv.org/abs/1611.06256
Empiezo vuelta 1 :10:57:41
scrapig google https://scholar.google.com/scholar?start=10&q=advantage+actor+critic+source%3Aarxiv&oq=
scraping a https://arxiv.org/abs/1607.07086
<Response [200]>
Empiezo vuelta 0 :10:57:44
scrapig google https://scholar.google.com/scholar?q=advantage+actor+critic+source%3Aspringer&oq=
scraping a https://link.springer.com/article/10.1007/s11276-019-02225-x
Empiezo vuelta 1 :10:57:47
scrapig google https://scholar.google.com/scholar?start=10&q=advantage+actor+critic+source%3Aspringer&oq=
scraping a https://link.springer.com/chapter/10.1007/978-3-319-71682-4_5
<Response [200]>
Empiezo vuelta 0 :10:58:12
scrapig google https://scholar.google.com/scholar?q=advantage+actor+critic+source%3ASSRN&oq=
scraping a https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4021959
Empiezo vuelta 1 

In [133]:
for paper in papers:
    print(paper)
    break

{'fuente': 'cambridge', 'titulo': ['Machine learning in mental health: a scoping review of methods and applications'], 'autores': ['Adrian B. R. Shatte', 'Delyse M. Hutchinson', 'Samantha J. Teague'], 'abstract': ['This paper aims to synthesise the literature on machine learning (ML) and big data applications for mental health, highlighting current research and applications in practice.', "We employed a scoping review methodology to rapidly map the field of ML in mental health. Eight health and information technology research databases were searched for papers covering this domain. Articles were assessed by two reviewers, and data were extracted on the article's mental health application, ML technique, data type, and study results. Articles were then synthesised via narrative review.", "Three hundred papers focusing on the application of ML to mental health were identified. Four main application domains emerged in the literature, including: (i) detection and diagnosis; (ii) prognosis, 

In [134]:
for i in papers:
    print(i["titulo"])

['Machine learning in mental health: a scoping review of methods and applications']
[]


In [135]:
try:
    for i in papers:
        print(i["titulo"])
except:pass
try:
    for i in papers:
        print(i["autores"])
except:pass
try:    
    for i in papers:
        print(i["abstract"])
except:pass
try:
    for i in papers:
        print(i["clase_primaria"])
except:pass
try: 
    for i in papers:
        print(i["n_citaciones"])
except:pass

['Machine learning in mental health: a scoping review of methods and applications']
[]
['Adrian B. R. Shatte', 'Delyse M. Hutchinson', 'Samantha J. Teague']
[]
['This paper aims to synthesise the literature on machine learning (ML) and big data applications for mental health, highlighting current research and applications in practice.', "We employed a scoping review methodology to rapidly map the field of ML in mental health. Eight health and information technology research databases were searched for papers covering this domain. Articles were assessed by two reviewers, and data were extracted on the article's mental health application, ML technique, data type, and study results. Articles were then synthesised via narrative review.", "Three hundred papers focusing on the application of ML to mental health were identified. Four main application domains emerged in the literature, including: (i) detection and diagnosis; (ii) prognosis, treatment and support; (iii) public health, and; (iv)